# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [23]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
spaceship.dropna(inplace=True)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [9]:
#your code here
# One-hot encode categorical columns
df_encoded = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin'], drop_first=True)

# Drop non-numeric columns that are not useful for modeling
df_encoded.drop(columns=['PassengerId', 'Name'], inplace=True)

# Initialize the scaler
scaler = StandardScaler()

# List of numerical features to scale
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Scale the numerical features
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

In [10]:
# Separate features and target variable
X = df_encoded.drop(columns=['Transported'])
y = df_encoded['Transported'].astype(int)

# Initialize SelectKBest with a statistical test
selector = SelectKBest(score_func=f_classif, k=10)

# Fit and transform the data to select the best features
X_new = selector.fit_transform(X, y)

# Get the selected feature indices
selected_features = selector.get_support(indices=True)

# Get the selected feature names
selected_feature_names = X.columns[selected_features]
print("Selected features:")
print(selected_feature_names)

Selected features:
Index(['Age', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck', 'HomePlanet_Europa',
       'CryoSleep_True', 'Destination_TRAPPIST-1e', 'VIP_True',
       'Cabin_G/943/S'],
      dtype='object')


**Perform Train Test Split**

In [11]:
#your code here
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y, test_size=0.2, random_state=42
)

# Display the shapes of the resulting datasets
print("Training features shape:", X_train.shape)
print("Test features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Test target shape:", y_test.shape)

Training features shape: (5284, 10)
Test features shape: (1322, 10)
Training target shape: (5284,)
Test target shape: (1322,)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [14]:
#your code here
# Initialize a Bagging classifier with decision trees as base learners
bagging_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,     # Number of models to train
    max_samples=1.0,      # Proportion of samples to draw
    bootstrap=True,       # True for bagging, False for pasting
    n_jobs=-1,            # Use all available cores
    random_state=42
)

# Train the bagging classifier
bagging_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = bagging_clf.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Bagging Classifier Accuracy: {accuracy:.2f}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging Classifier Accuracy: 0.78


In [15]:
# Initialize a Pasting classifier with decision trees as base learners
pasting_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,     # Number of models to train
    max_samples=1.0,      # Proportion of samples to draw
    bootstrap=False,      # False for pasting (no replacement)
    n_jobs=-1,            # Use all available cores
    random_state=42
)

# Train the pasting classifier
pasting_clf.fit(X_train, y_train)

# Predict on the test data
y_pred_pasting = pasting_clf.predict(X_test)

# Evaluate the accuracy
accuracy_pasting = accuracy_score(y_test, y_pred_pasting)
print(f"Pasting Classifier Accuracy: {accuracy_pasting:.2f}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Pasting Classifier Accuracy: 0.75


- Random Forests

In [19]:
#your code here
# Initialize Random Forest classifier
rf_clf = RandomForestClassifier(
    n_estimators=100,    # Number of trees in the forest
    max_depth=None,      # Allow trees to grow until all leaves are pure or until they contain less than min_samples_split samples
    random_state=42,
    n_jobs=-1            # Use all available cores
)

# Train the Random Forest classifier
rf_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_clf.predict(X_test)

# Evaluate the accuracy and provide a classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Classifier Accuracy: {accuracy:.2f}")

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Random Forest Classifier Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.77      0.78       653
           1       0.78      0.80      0.79       669

    accuracy                           0.79      1322
   macro avg       0.79      0.78      0.79      1322
weighted avg       0.79      0.79      0.79      1322



- Gradient Boosting

In [22]:
#your code here
# Initialize Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(
    n_estimators=100,    # Number of boosting stages to be run
    learning_rate=0.1,   # Controls contribution of each tree
    max_depth=3,         # Maximum depth of individual regression estimators
    random_state=42
)

# Train the Gradient Boosting classifier
gb_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = gb_clf.predict(X_test)

# Evaluate the accuracy and provide a classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Gradient Boosting Classifier Accuracy: {accuracy:.2f}")

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Gradient Boosting Classifier Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.73      0.77       653
           1       0.76      0.84      0.80       669

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



- Adaptive Boosting

In [24]:
#your code here
ada_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),  # Decision stump
    n_estimators=50,  # Number of weak learners
    learning_rate=1.0,
    random_state=42
)

# Train the AdaBoost classifier
ada_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = ada_clf.predict(X_test)

# Evaluate the accuracy and provide a classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"AdaBoost Classifier Accuracy: {accuracy:.2f}")

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoost Classifier Accuracy: 0.78
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.75      0.77       653
           1       0.77      0.81      0.79       669

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322



Which model is the best and why?

In [ ]:
#comment here